---
## BeautifulSoup
```
    파이썬에서 HTML 코드를 파싱하기 위해선 BeautifulSoup을 이용해야 한다
    HTML 코드를 BeautifulSoup타입으로 바꿔 처리한다
```

In [1]:
html_code = """<!DOCTYPE html>
<html>
<head>
    <title>Sample Website</title>
</head>
<body>
<h2>HTML 연습!</h2>

<p>이것은 첫 번째 문단입니다.</p>
<p>이것은 두 번째 문단입니다!</p>

<ul>
    <li>커피</li>
    <li>녹차</li>
    <li>우유</li>
</ul>

<img src='https://i.imgur.com/bY0l0PC.jpg' alt="coffee"/>
<img src='https://i.imgur.com/fvJLWdV.jpg' alt="green-tea"/>
<img src='https://i.imgur.com/rNOIbNt.jpg' alt="milk"/>

</body>
</html>"""

In [3]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup

In [5]:
#  BeautifulSoup 타입으로 변환
soup = BeautifulSoup(html_code, 'html.parser') # html을 파싱한다는 의미로 html.parser같이 보내주기

# type 확인
type(soup)

bs4.BeautifulSoup

#### BeautifulSoup타입에는 .select() 메소드를 사용할 수 있다.
#### 괄호안에 CSS선택자를 넣고 특정 HTML 태그만 선택할 수 있다.

In [16]:
# 모든 <li> 태그 선택하기
li_tags = soup.select('li')

print(li_tags)
# 리스트로 변환되어 하나하나 요소에 접근할 수 있다.
print(li_tags[0])

# 타입을 확인해보면 element.Tag로 확인된다.
print(type(li_tags[0]))

# element.Tag의 기능 중 하나는 텍스트 추출이다. 뒤에 .text라고 붙혀주면, 텍스트만 꺼내 확인할 수 있다.
print(li_tags[2].text)

# 모두 뽑아내려면 반복문을 사용한다.
beverage_names=[]

for li in li_tags:
    beverage_names.append(li.text)
    
print(beverage_names)

[<li>커피</li>, <li>녹차</li>, <li>우유</li>]
<li>커피</li>
<class 'bs4.element.Tag'>
우유
['커피', '녹차', '우유']


---
#### BeautifulSoup태그에 .text를 붙여 텍스트를 추출하는 것처럼 태그의 속성값을 추출할 수 있다.
#### img 태그의 src 속성에는 일반적으로 이미지 주소가 저장되어 있다. 이를 추출해보자.

In [17]:
img_tags = soup.select('img')
img_tags

[<img alt="coffee" src="https://i.imgur.com/bY0l0PC.jpg"/>,
 <img alt="green-tea" src="https://i.imgur.com/fvJLWdV.jpg"/>,
 <img alt="milk" src="https://i.imgur.com/rNOIbNt.jpg"/>]

In [20]:
# 이미지 역시 배열로 받아온다. 이를 항목 하나하나에 접근할 수 있다.
print(img_tags[0])

# 여기서 src속성에 접근하려면 ['src']를 붙혀준다
print(img_tags[0]['src'])

# 위와 마찬가지로 for문으로 뽑아낼 수 있다.
src_list = []

for src in img_tags:
    src_list.append(src['src'])

print(src_list)

<img alt="coffee" src="https://i.imgur.com/bY0l0PC.jpg"/>
https://i.imgur.com/bY0l0PC.jpg
['https://i.imgur.com/bY0l0PC.jpg', 'https://i.imgur.com/fvJLWdV.jpg', 'https://i.imgur.com/rNOIbNt.jpg']


---
### 파싱 실습
* 음악사이트의 인기 아티스트를 추출해보기

In [24]:
# 1. HTML 코드 받아오기
import requests

# HTML 코드 받아오기
response = requests.get("https://workey.codeit.kr/music/index")

print(response.text) # html 을 보려면 .text 사용

In [35]:
# 2. BeautifulSoup타입으로 교체
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# print(soup)

In [36]:
# 인기아티스트를 찾아보니 h3 태그에 클래스 명은 'popular_title' 이다.
# 그 내부에 있는 ul 태그의 선택자를 통해 파싱해본다.
popular_singers_html = soup.select('.popular__order li')
# 변수에 인기 아티스트 10명과 html문법들을 리스트로 잘 저장되어있다.
# 가수 텍스트만 추출해낸다.
popular_singers = []
for singer in popular_singers_html:
    popular_singers.append(singer.text.strip()) # strip을 사용하여 공백과 개행 문자까지 지울 수 있다.
popular_singers

['1 아이유 (IU)',
 '2 방탄소년단',
 '3 Red Velvet (레드벨벳)',
 '4 IKON',
 '5 멜로망스',
 '6 다비치',
 '7 윤딴딴',
 '8 수지 (SUZY)',
 '9 김동률',
 '10 폴킴']

---
### 필요한 페이지 만큼만 가져오기
* 이케아 desk 검색해보기  

```
필요한 만큼만 가져오는 방법
1. 파싱으로 페이지 개수 확인하기 (사이트에 나와있는경우)
2. 일단 가져오고 확인해보기
```

In [67]:
# 1. 파싱으로 페이지 개수 확인하기
import requests
from bs4 import BeautifulSoup

# HTML 코드 받아오기
response = requests.get("https://www.ikea.com/kr/ko/search/?query=desk")

# BeautifulSoup 타입으로 변환
soup = BeautifulSoup(response.text, 'html.parser')

# 마지막 <a> 태그의 텍스트 추출하기 (인덱스의 -1 은 마지막 항목을 가르킨다.) 
page_length = soup.select('.pagination a')[-1].text

desk_page_htmls = []
# page_length = 13
for num in range(1, int(page_length)+1):
    desk_page_htmls.append(requests.get("https://www.ikea.com/kr/ko/search/?query=desk&pageNumber="+str(num)).text)

print(len(desk_page_htmls))
# print(desk_page_htmls[0])

13


https://www.ikea.com/kr/ko/search/?query=desk&pageNumber=1   
https://www.ikea.com/kr/ko/search/?query=desk&pageNumber=833

```
제품이 존재하는 페이지와 존재하지 않는 페이지를 비교하는방법이다.
문제에서는 prodName이라는 클래스의 존재 유무로 검사를 해보았다.
```

In [71]:
# 2. 일단 가져오고 확인해보기.
import requests
from bs4 import BeautifulSoup

page_num = 1
pages = []
while True:
    # HTML 코드 받아오기
    response = requests.get("https://www.ikea.com/kr/ko/search/?query=desk&pageNumber="+str(page_num))

    # BeautifulSoup 타입으로 변환하기
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # .prodName클래스의 유무 확인
    if(len(soup.select('.prodName')) > 0):
        pages.append(soup.text)
        page_num+=1
    else:
        break;
print(len(pages))
# print(pages[0])

13


---
## 웹 크롤링 후 DataFrame 으로 정형하기

In [97]:
import requests
from bs4 import BeautifulSoup

page_num = 1
records = []

while True:
    # HTML 코드 받아오기
    response = requests.get("https://www.ikea.com/kr/ko/search/?query=desk&pageNumber="+str(page_num))

    # BeautifulSoup 타입으로 변환하기
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # .prodName클래스의 유무 확인
    if(len(soup.select('.prodName')) > 0):
        product_names = soup.select('.prodName')
        product_prices = soup.select('.prodPrice')
        product_imgs = soup.select('.prodImg')
        page_num+=1
        for i in range(0, len(soup.select('.prodName'))):
            record = []
            record.append(product_names[i].text)
            record.append(product_prices[i].text.strip())
            record.append("https://www.ikea.com" + product_imgs[i].get('src'))
            records.append(record)        
    else:
        break;
print(len(records))
#print(records)

# DataFrame 만들기
import pandas as pd

df = pd.DataFrame(data=records, columns=['제품명', '가격', '제품 이미지 주소'])
df.head()

305


,제품명,가격,제품 이미지 주소
0,BRÄDA 브레다,"₩ 4,900 /개",https://www.ikea.com/kr/ko/images/products/bra...
1,ISBERGET 이스베리에트,"₩ 2,500 /개",https://www.ikea.com/kr/ko/images/products/isb...
2,FREDDE 프레데,"₩ 279,000 /개",https://www.ikea.com/kr/ko/images/products/fre...
3,BERGENES 베리에네스,"₩ 2,900 /개",https://www.ikea.com/kr/ko/images/products/ber...
4,BEKANT 베칸트,"₩ 189,000 /개",https://www.ikea.com/kr/ko/images/products/bek...
